This notebook will be an attempt to mirror some of the changes that Dan went through after talking with the creator of uclust.  in particular, I will be using the uclust merging algorithym and a more stringent quality filter.  Essentially, the uclust merger acts to decrease the creation of false OTUs.  It does this by giving weight to the higher of the two reads(foreward vs reverse) when determining a base pair.  If there is aggreement, the quality score goes up-if not, the higher quality read has extra weight, but the final quality score is lower.  I think I can start with my already compled grass foreward and reverse fastq files (grassR1.fastq and grassR2.fastq)

In [1]:
pwd

/Users/grahambailes/grass_endophyte_community


In [2]:
mkdir alt_biom

In [5]:
cp ./grass_biom/grassR1.fastq ./alt_biom
cp ./grass_biom/grassR2.fastq ./alt_biom

In [2]:
cd ./alt_biom
pwd

/Users/grahambailes/grass_endophyte_community/alt_biom


## in ACISS
##trim reads at same length as the original iteration of the biom table.  this is done with the FASTX_toolkit
fastx_trimmer -l 255 -i grassR1.fastq grassR1_short.fastq
fastx_trimmer -l 220 -i grassR2.fastq grassR2_short.fastq

# merge pairs

At this point, I'll be using the usearch algorithm to do the merging.
The command fastq_mergepairs takes a foreward and reverse file and returns a merged fastq file.  the notrunclables option keeps lables.  For our run, 

In [ ]:
##again in aciss - I've made a directory 
mkdir alt_biom
cd alt_biom
usearch -fastq_mergepairs /home11/gbailes/endophyte_community/grassR1_short.fastq -reverse /home11/gbailes/endophyte_community/grassR2_short.fastq -fastqout grass_trimmed_merged.fastq -notrunclabels -report   


### merge pairs output

usearch v8.1.1803_i86linux64, 74.2Gb RAM, 12 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: ronh@molbio.uoregon.edu

04:40 925Mb  100.0% 97.3% merged 
  11033436  Pairs (11.0M)   
  
  10735444  Merged (10.7M, 97.30%)
 
 5462314  Alignments with zero diffs (49.51%)
 
 1795  Fwd tails Q <= 2 trimmed (0.02%)
 
 3099  Rev tails Q <= 2 trimmed (0.03%)
 
 297992  No alignment found (2.70%)
 
 0  Alignment too short (< 16) (0.00%)
 
 8141236  Staggered pairs (73.79%) merged & trimmed
 
 209.71  Mean alignment length
 
 244.01  Mean merged read length
 
 2.00  Mean fwd expected errors
 
 1.77  Mean rev expected errors
 
 1.12  Mean merged expected errors

In [ ]:
fastx_quality_stats -Q33 -i grass_trimmed_merged.fastq -o grass_trimmed_merged_quality.txt
#(ACISS has older version of fastx as of time of writing, so we in "-Q33" flag to
#tell fastx that we are dealing with Sanger-type quality encoding, or "Phred+33")

In [3]:
./quality_plot.sh -i grass_trimmed_merged_quality.txt -o grass_trimmed_merged_quality.png

bash: ./quality_plot.sh: No such file or directory


: 127

![merged quality](/notebooks/grass_endophyte_community/alt_biom/grass_trimmed_merged_quality.png "ShowMyImage")

# Usearch quality filter

the usearch pipeline further suggests quality filtering of reads.  I defer to Dan's knowledge on using this command.  Looks like he used expected error approach. We can set error cutoff of 1% of all bases in a read, meaning that a read of length 400 bp is thrown out if it likely contains 4 or more erroneous bases. I think this is permissable, given our OTU clustering will ultimately be done at 95% similarity.

http://drive5.com/usearch/manual/exp_errs.html

In [ ]:
usearch -fastq_filter grass_trimmed_merged.fastq -fastq_maxee_rate .01 -fastqout grass_filtered.fastq -notrunclabels

usearch v8.1.1861_i86linux64, 74.2Gb RAM, 12 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: ronh@molbio.uoregon.edu

01:11 857Mb  100.0% Filtering, 88.4% passed
  10735444  FASTQ recs (10.7M)             
   9485630  Converted (9.5M, 88.4%)
   
looks like about 11.6% of my reads were filtered out.   

## convert from fastq to fasta

In [ ]:
#use fastx_toolkit to convert to fasta
fastq_to_fasta -n -Q33 -i grass_filtered.fastq -o  grass.fasta

# -n keep sequences with unknown (N) nucleotides, -Q33 

## floating primers
sequences may often include multiple primer occurances, sometimes sprinkled within the sequence.  these should be removed according to http://onlinelibrary.wiley.com/doi/10.1002/ece3.1107/epdf (balint et al. 2014)


In [ ]:
#check for floating primers
grep CTTGGTCATTTAGAGGAAGTAA grass.fasta | wc -l #forward (ITS1)
grep GCTGCGTTCTTCATCGATGC grass.fasta | wc -l #reverse (ITS2)
# 43, 28

In [ ]:
#using a python script that dan wrote to remove primer sequences from the read files
./floatingprimers.py  grass_paired_clipped.fasta  grass_paired_clipped_defloat.fasta CTTGGTCATTTAGAGGAAGTAA GCTGCGTTCTTCATCGATGC 


In [ ]:
#To count reads in these we need to grep the sequence identifier line, 
#instead of just using the 'wc -l' command:
grep '\<M' grass.fasta | wc -l
grep '\<M' grass_defloat.fasta | wc -l

grass.fasta: 9485630

grass_defloat.fasta: 9485318

312 reads removed (0.003%) for the presence of floating primer relics

## chimera checking

We'll use the uchime algorithm, another step in the uparse/usearch pipeline. This is actually just the first of two checks for chimeras, the other being part of the otu clustering.  I've already downloaded an iteration of the uchime reference


In [ ]:
usearch -uchime_ref grass_defloat.fasta -db /home11/gbailes/alt_biom/uchime_sh_refs_dynamic_develop_985_01.01.2016.ITS1.fasta -nonchimeras grass_notchim.fasta -strand plus -uchimeout grass_chim_log.txt -notrunclabels


grass_defloat.fasta: 9485318

grass_notchim.fasta: 9432696

52,622 reads removed (~0.5% of reads)

## creating simple lables

using the sed command, I'll relable the reads to have a simple identifier.  I'll also include a file which connects the new lables to the original sample names (grass_illumina_lables.csv)

In [ ]:
sed s/^\>.*[0-9]\ //g grass_notchim.fasta | sed s/1:[YN]:0:/\>/g | sed '/^>/ s/$/grass/g' > grass_relab.fasta

## trimming reads to ITS1

apparently, inorder to best cluster otus, you first need to trim out any highly-conserved regions of the ITS (i.e. the SSU and 5.8 regions)  

In [ ]:
./fasta_remove_linebreaks.py grass_relab.fasta grass_relab_nolb.fasta ## remove linebreaks from fasta files
./subset_fasta.py grass_relab_nolb.fasta 100 grass_sub.fasta ## subset reads to examine proportion of ssu and 5.8 regions
 ./ITSx -i grass_sub.fasta -o grass_sub --reset T -- allow_single_domain -t F 
 
# ITSx -- Identifies ITS sequences and extracts the ITS region
#by Johan Bengtsson-Palme et al., University of Gothenburg
#Version: 1.0.11
#-----------------------------------------------------------------
#Sat Dec 10 15:13:01 2016 : Preparing HMM database (should be quick)...
#Sat Dec 10 15:13:01 2016 : Checking and handling input sequence data (should not take long)...
#Sat Dec 10 15:13:01 2016 : Comparing sequences to HMM database (this may take a long while)...
#    Sat Dec 10 15:13:06 2016 : Fungi analysis of main strand finished.
#    Sat Dec 10 15:13:11 2016 : Fungi analysis of complementary strand finished.
#Sat Dec 10 15:13:11 2016 : Analysing results of HMM-scan (this might take quite some time)...
#Sat Dec 10 15:13:11 2016 : Extraction finished!
##-----------------------------------------------------------------
#Thank you for using ITSx!
#Please report bugs or unsupported lineages to itsx@microbiology.se

In [6]:
head -50 grass_sub.positions.txt; tail -50 grass_sub.positions.txt 

5grass	295 bp.	SSU: 1-46	ITS1: 47-265	5.8S: No end	ITS2: Not found	LSU: Not found	Broken or partial sequence, only partial 5.8S! 
7grass	229 bp.	SSU: 1-46	ITS1: 47-199	5.8S: No end	ITS2: Not found	LSU: Not found	Broken or partial sequence, only partial 5.8S! 
8grass	270 bp.	SSU: 1-46	ITS1: 47-240	5.8S: No end	ITS2: Not found	LSU: Not found	Broken or partial sequence, only partial 5.8S! 
12grass	285 bp.	SSU: 1-46	ITS1: 47-255	5.8S: No end	ITS2: Not found	LSU: Not found	Broken or partial sequence, only partial 5.8S! 
17grass	254 bp.	SSU: 1-46	ITS1: 47-224	5.8S: No end	ITS2: Not found	LSU: Not found	Broken or partial sequence, only partial 5.8S! 
18grass	229 bp.	SSU: 1-46	ITS1: 47-199	5.8S: No end	ITS2: Not found	LSU: Not found	Broken or partial sequence, only partial 5.8S! 
20grass	217 bp.	SSU: 1-46	ITS1: 47-187	5.8S: No end	ITS2: Not found	LSU: Not found	Broken or partial sequence, only partial 5.8S! 
24grass	225 bp.	SSU: 1-46	ITS1: 47-195	5.8S: No end	ITS2: Not found	LSU: Not found	Bro

Looks like for most sequences, the SSU is 46 bp, and the 5.8S gene is 30.  Within this subset, there is more variation within the 5.8S - these values range from 20 to 70 or so.  However, since the majority are centered at 30, I'll trim 46 bp from the , and 30bp from 

In [27]:
fastx_trimmer -f 47 -i grass_relab_nolb.fasta | fastx_trimmer -t 30 -o grass.ITS1.fasta

## pick OTUs and representative sequences



In [ ]:
## Transfer fasta with ITS1 only to aciss. for some reason, sftp was only transfering a fraction of my
## file, so I chose to use the scp comand.  seems to have done the trick
 scp ./grass.ITS1.fasta gbailes@aciss.uoregon.edu:/home11/gbailes/biom_dec_16
 
 #remember to load appropriate modules on aciss:
 module load usearch/8.1.1803
 module load python/3.5.2

In [ ]:
## dereplicate sequences
usearch -derep_fulllength ./grass.ITS1.fasta -fasout ./grass_ITS1_derep.fasta -sizeout


usearch v8.1.1803_i86linux64, 74.2Gb RAM, 12 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: ronh@molbio.uoregon.edu

00:24 2.3Gb  100.0% Reading ./grass.ITS1.fasta
00:29 5.0Gb 9417846 seqs, 1182213 uniques, 900911 singletons (76.2%)
00:29 5.0Gb Min size 1, median 1, max 727417, avg 7.97
00:29 5.0Gb  100.0% Writing 
00:36 3.2Gb  100.0% Writing ./grass_ITS1_derep.fasta

looks like we have 9,417846 sequences, 1,182,213 uniques, 900,911 singletons (76.2%)

In [ ]:
## sort by size
usearch -sortbysize ./grass_ITS1_derep.fasta -fasout ./grass_ITS1_sorted.fasta -minsize 2

usearch v8.1.1803_i86linux64, 74.2Gb RAM, 12 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: ronh@molbio.uoregon.edu

00:03 342Mb  100.0% Reading ./grass_ITS1_derep.fasta
00:03 308Mb Getting sizes                           
00:04 318Mb Sorting 281302 sequences
00:05 319Mb  100.0% Writing output

In [ ]:
## look at total unique sequences, vs singletons.  
grep '>' grass_ITS1_derep.fasta | wc -l
grep '>' grass_ITS1_sorted.fasta | wc -l

1182213,
281302
looks like lots of singletons!


# cluster to 97% radius

I think that the default clustering uses minimum distance/single linkage to create clusters - This means that within a cluster, un-grouped sequences are compared with a single centroid and rejected with a similarity > 0.3.  This means that within a single cluster, there may be sequences that are more dissimilar than 97%, solong as they agree with the centroid.  This type of clustering distance results in fewer, larger clusters than other methods (maximum distance/complete linkage)


Dan wrote a python script that will  to our clusters.  Apparently the 'cluster-_smallmem' clustering algorithm 
(as opposed to the 'usearch_cluster_otus') doesn't automatically add the '

In [ ]:
## cluster using uclust 
usearch -cluster_smallmem ./grass_ITS1_sorted.fasta -id 0.97 -centroids ./otus_97_uclust.fasta -sizein -sizeout -sortedby size

usearch v8.1.1803_i86linux64, 74.2Gb RAM, 12 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: ronh@molbio.uoregon.edu

00:09  53Mb  100.0% 3922 clusters, max size 1179223, avg 2171.6
00:10  53Mb  100.0% Writing centroids to ./otus_97_uclust.fasta
                                                               
      Seqs  281302 (281.3k)
  Clusters  3922
  Max size  1179223 (1.2M)
  Avg size  2171.6
  Min size  2
Singletons  0, 0.0% of seqs, 0.0% of clusters
   Max mem  53Mb
      Time  34.0s
Throughput  8273.6 seqs/sec.

Okay, looks like our 281302 sequences clustered to 3922 clusters.  This makes me think that there were no more chimeras detected durring this process

In [ ]:
## add otu tags to sequences
./addOTUtags.py ./otus_97_uclust.fasta OTU ./otus_97_uclust_relabel.fasta

In [4]:
# there was a new release of the UNITE database, so lets see how they compare
usearch -tax_stats utax_ref_class_v7_utax.fasta 
usearch -tax_stats utax_ref_class.fasta

usearch v8.1.1861_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: bailes.graham@gmail.com

00:00  79Mb  100.0% Reading utax_ref_class_v7_utax.fasta
00:01  50Mb  100.0% Initialize taxonomy data            
00:01  51Mb  100.0% Building name table     
00:01  51Mb 12837 names, tax levels min 3, avg 5.8, max 7
    Seqs  LevelPath
   18264  Domain,Phylum,Class,Order,Family,Genus
    9140  Domain,Phylum,Class,Order,Family,Genus,Species
    6269  Domain,Phylum,Class,Order,Family
    3367  Domain,Phylum,Class,Order
    1236  Domain,Phylum,Class

  Names  Level
      1  Domain
      7  Phylum
     51  Class
    175  Order
    533  Family
   2929  Genus
usearch v8.1.1861_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: bailes.graham@gmail.com

00:01  82Mb  100.0% Reading utax_ref_class.fasta
00:01  53Mb  100.0

In [5]:
## assign taxonomy
##since I created the utax reference database on my labtop (32 bit), this also was run on the laptop
## first transfer relabled uclust fasta back to personal computer:
# scp gbailes@aciss.uoregon.edu:/home11/gbailes/biom_dec_16/otus_97_uclust_relabel.fasta /Users/grahambailes/grass_endophyte_community/alt_biom/
usearch -utax otus_97_uclust_relabel.fasta -db utax_ref_class_v7.udb -strand both -fastaout otus_97_uclust_ass_tax.fasta

usearch v8.1.1861_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: bailes.graham@gmail.com

00:01 123Mb  100.0% Rows
00:01 123Mb Read taxonomy info...done.
00:01 125Mb Reading pointers...done.
00:01 129Mb Reading db seqs...done.
00:03 162Mb  100.0% 3922 seqs, 74.4% at phylum, 13.0% genus (P > 0.90)


usearch v8.1.1861_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: bailes.graham@gmail.com

00:01 123Mb  100.0% Rows
00:01 123Mb Read taxonomy info...done.
00:01 125Mb Reading pointers...done.
00:01 129Mb Reading db seqs...done.
00:03 162Mb  100.0% 3922 seqs, 74.4% at phylum, 13.0% genus (P > 0.90)

## create 97% radius biom table

In [6]:
usearch -usearch_global grass.ITS1.fasta -db otus_97_uclust_ass_tax.fasta -strand both -id 0.97 -biomout grass_otu_uclust_97.biom

usearch v8.1.1861_i86osx32, 4.0Gb RAM (8.6Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: bailes.graham@gmail.com

00:00 4.3Mb  100.0% Reading otus_97_uclust_ass_tax.fasta
00:00 2.9Mb  100.0% Masking                             
00:00 3.8Mb  100.0% Word stats
00:00 3.8Mb  100.0% Alloc rows
00:00 6.1Mb  100.0% Build index
24:33  42Mb  100.0% Searching, 99.9% matched 
9388226 / 9417846 mapped to OTUs (99.7%)    
24:33  42Mb Writing grass_otu_uclust_97.biom
24:33  42Mb Writing grass_otu_uclust_97.biom ...done.


(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: bailes.graham@gmail.com


00:00 4.3Mb  100.0% Reading otus_97_uclust_ass_tax.fasta
00:00 2.9Mb  100.0% Masking                             
00:00 3.8Mb  100.0% Word stats
00:00 3.8Mb  100.0% Alloc rows
00:00 6.1Mb  100.0% Build index
24:33  42Mb  100.0% Searching, 99.9% matched 
9388226 / 9417846 mapped to OTUs (99.7%)    
24:33  42Mb Writing grass_otu_uclust_97.biom
24:33  42Mb Writing grass_otu_uclust_97.biom ...done.


## format biome table

clean-up of the biome table created through the usearch algorithms.  Downstream programs and packages cannot read as is...

In [8]:
sed '/taxonomy/ s/([0-1]\.[0-9]*)//g' grass_otu_uclust_97.biom |\
sed -E 's/("taxonomy")(:")/\1:[/' |\
sed -E 's/"}}/,]}}/' |\
sed -E '/taxonomy/ s/(d:)([^,]*)/"k__\2"/' |\
sed -E '/taxonomy/ s/(p:)([^,]*)/"p__\2"/' |\
sed -E '/taxonomy/ s/(c:)([^,]*)/"c__\2"/' |\
sed -E '/taxonomy/ s/(o:)([^,]*)/"o__\2"/' |\
sed -E '/taxonomy/ s/(f:)([^,]*)/"f__\2"/' |\
sed -E '/taxonomy/ s/(g:)([^,]*)/"g__\2"/' |\
sed -E '/taxonomy/ s/(s:)([^,]*)/"s__\2"/' |\
sed -E '/taxonomy/ s/,]}}/]}}/' > grass_otu_uclust_97_relab.biom

In [9]:
# take another look at a row 
grep 'rows' -A 10 grass_otu_uclust_97_relab.biom 

	"rows":[
		{"id":"OTU2:5grass", "metadata":{"taxonomy":["k__Fungi","p__Ascomycota","c__Sordariomycetes","o__Hypocreales","f__Hypocreales_fam_Incertae_sedis","g__Sarocladium"]}},
		{"id":"OTU1:5grass", "metadata":{"taxonomy":["k__Fungi","p__Ascomycota","c__Dothideomycetes","o__Capnodiales","f__Davidiellaceae","g__Davidiella","s__Davidiella_tassiana_SH216250.07FU"]}},
		{"id":"OTU16:5grass", "metadata":{"taxonomy":["k__Fungi","p__Ascomycota","c__Dothideomycetes","o__Capnodiales"]}},
		{"id":"OTU263:5grass", "metadata":{"taxonomy":["k__Fungi","p__Ascomycota","c__Leotiomycetes","o__Helotiales","f__Dermateaceae","g__Dermea","s__Dermea_ariae_SH201690.07FU"]}},
		{"id":"OTU4:5grass", "metadata":{"taxonomy":["k__Fungi","p__Ascomycota","c__Sordariomycetes","o__Sordariomycetidae_ord_Incertae_sedis","f__Glomerellaceae","g__Glomerella"]}},
		{"id":"OTU1265:13grass", "metadata":{"taxonomy":["k__Fungi","p__Ascomycota","c__Sordariomycetes","o__Sordariomycetidae_ord_Incertae_sedis","f__Glomerellaceae

In [3]:
# and at columns
grep 'columns' -A 40  grass_otu_uclust_97_relab.biom 

	"columns":[
		{"id":"5grass", "metadata":null},
		{"id":"6grass", "metadata":null},
		{"id":"7grass", "metadata":null},
		{"id":"8grass", "metadata":null},
		{"id":"9grass", "metadata":null},
		{"id":"10grass", "metadata":null},
		{"id":"11grass", "metadata":null},
		{"id":"12grass", "metadata":null},
		{"id":"13grass", "metadata":null},
		{"id":"14grass", "metadata":null},
		{"id":"15grass", "metadata":null},
		{"id":"16grass", "metadata":null},
		{"id":"17grass", "metadata":null},
		{"id":"18grass", "metadata":null},
		{"id":"19grass", "metadata":null},
		{"id":"20grass", "metadata":null},
		{"id":"21grass", "metadata":null},
		{"id":"22grass", "metadata":null},
		{"id":"23grass", "metadata":null},
		{"id":"24grass", "metadata":null},
		{"id":"25grass", "metadata":null},
		{"id":"26grass", "metadata":null},
		{"id":"27grass", "metadata":null},
		{"id":"28grass", "metadata":null},
		{"id":"29grass", "metadata":null},
		{"id":"30grass", "metadata":null},
		{"id":"31grass", "metadata":

In [11]:
# command to test whether the biom table is in correct format for downstream applications
biom validate-table -i grass_otu_uclust_97_relab.biom

Invalid format 'Biological Observation Matrix 1.0', must be '1.0.0'
'id' in {'id': '', 'metadata': {'taxonomy': ['k__Fungi', 'p__Basidiomycota', 'c__Agaricomycetes', 'o__Agaricales', 'f__Strophariaceae', 'g__Melanotus']}} appears empty
Bad value at idx 0: [0, 0, 39534]
Timestamp does not appear to be ISO 8601
The input file is not a valid BIOM-formatted file.


: 1

In [ ]:
# there are several small issues here - this also happened to Dan, where there was 
#1) an ID that was missing, 2) a 'bad value' at [0,0,11660], and the non-matching timestamp.
#I don't really care about the time-stamp issue, so I'll look into the other two:


In [12]:
# look for information on missing id
grep '"id":""' grass_otu_uclust_97.biom

		{"id":"", "metadata":{"taxonomy":"d:Fungi,p:Basidiomycota(0.9997),c:Agaricomycetes(0.9777),o:Agaricales(0.9801),f:Strophariaceae(0.8777),g:Melanotus(0.8088)"}},


In [13]:
# find the associated otu in the otu assignment fasta file
grep 'd:Fungi,p:Basidiomycota(0.9997),c:Agaricomycetes(0.9777),o:Agaricales(0.9801),f:Strophariaceae(0.8777),g:Melanotus(0.8088)' otus_97_uclust_ass_tax.fasta 

>OTU2489:6grass;size=6;tax=d:Fungi,p:Basidiomycota(0.9997),c:Agaricomycetes(0.9777),o:Agaricales(0.9801),f:Strophariaceae(0.8777),g:Melanotus(0.8088);


In [ ]:
# its a bit odd that three otus had 1) the same confidence for each taxonomic level, and 2) such low confidence levels in general
# as I saw when grepping the columns, position 0,0,11660 is from sample grass5, so I'll insert this otu (OTU697:5grass) in the empty assignment

In [14]:
sed -i' ' '/"id":""/ s/"id":"",/"id":"6grass",/' grass_otu_uclust_97_relab.biom

In [15]:
grep '"id":""' grass_otu_uclust_97_relab.biom | wc -l


       0


In [16]:
## check out the validate command agin
biom validate-table -i grass_otu_uclust_97_relab.biom

Invalid format 'Biological Observation Matrix 1.0', must be '1.0.0'
Bad value at idx 0: [0, 0, 39534]
Timestamp does not appear to be ISO 8601
The input file is not a valid BIOM-formatted file.


: 1

In [17]:
## lets see if we can get rid of the bad value
grep \\[0,0,39534\\] -A 10 -B 10 grass_otu_uclust_97_relab.biom

		{"id":"159grass", "metadata":null},
		{"id":"160grass", "metadata":null},
		{"id":"161grass", "metadata":null},
		{"id":"162grass", "metadata":null},
		{"id":"163grass", "metadata":null},
		{"id":"255grass", "metadata":null},
		{"id":"256grass", "metadata":null},
		{"id":"257grass", "metadata":null}
	],
	"data": [
		[0,0,39534],
		[0,1,12136],
		[0,2,6426],
		[0,3,17774],
		[0,4,11],
		[0,5,13],
		[0,6,13],
		[0,8,8],
		[0,9,9],
		[0,10,1],
		[0,11,7],


Not sure what the problem is...

In [18]:
# command to look at sumary of biom table
biom summarize-table -i grass_otu_uclust_97_relab.biom

Num samples: 162
Num observations: 3880
Total count: 9388226
Table density (fraction of non-zero values): 0.058

Counts/sample summary:
 Min: 1745.0
 Max: 111141.0
 Median: 60985.500
 Mean: 57952.012
 Std. dev.: 20558.843
 Sample Metadata Categories: None provided
 Observation Metadata Categories: taxonomy

Counts/sample detail:
255grass: 1745.0
16grass: 7101.0
11grass: 10084.0
10grass: 12928.0
15grass: 14654.0
14grass: 14871.0
163grass: 17788.0
154grass: 18483.0
131grass: 21487.0
47grass: 22870.0
9grass: 23698.0
52grass: 23988.0
153grass: 24041.0
51grass: 25215.0
107grass: 27658.0
130grass: 27948.0
142grass: 28050.0
12grass: 29453.0
49grass: 29483.0
46grass: 29654.0
23grass: 30332.0
127grass: 33575.0
13grass: 33746.0
35grass: 34619.0
95grass: 34856.0
106grass: 35863.0
119grass: 37771.0
90grass: 38554.0
22grass: 38674.0
50grass: 38715.0
71grass: 39217.0
73grass: 39698.0
125grass: 39924.0
129grass: 40718.0
76grass: 40747.0
34grass: 44080.0
126grass: 44526.0
162grass: 45174.0
94grass: 45

## add sample name metadata to .97 biome table

In [19]:
# I've created a csv file which contains the sample Id and names.  I converted this into a tab-delimited file (.txt),
# which I used to add metadata.  for some reason the notebook won't print the contents, but if finally worked
head meta_data.txt

257grass	162	wood_pos_its	Pos_PCR	Control																																															

In [20]:
biom add-metadata -i grass_otu_uclust_97_relab.biom -o grass_97_wmeta.biom -m meta_data.txt --output-as-json

In [21]:
biom validate-table -i grass_97_wmeta.biom


The input file is a valid BIOM-formatted file.


# Exploration of Biome table

## 

I'll be using R to do most of the exploration.  I haven't taken the time to fix the install of R into the jupyter notebook, so I've made an accompanying R document to work through my exploration.  /Users/grahambailes/grass_endophyte_community/statistics/grass_97_biom.r



## convert .biom format into .txt 

It may be usefull to perform certain actions on the biom table outside of the R package phyloseq, and therefore I have decided to create a .txt version of my biom table.  These applications include species accumulation cureves, the use of the FUNGuild tool (https://github.com/UMNFuN/FUNGuild), etc.


In [4]:
## use QIIME tool to convert from .biom to .txt
## this is the .biom which I created through this pipeline.  
## At this point in time, I haven't done any variance stabilization methods, so this will be an exploration
## of the tools I wish to use.
## note - this is an otu table that was created prior to variance stabalization.

biom convert -i grass_97_wmeta.biom -o 97_otu_table.txt --to-tsv --header-key taxonomy

In [3]:
#Okay, I'll create an otu table from the variance stabalized biom (entitled grass_biom_vs)
biom convert -i grass_biom_vs.biom -o 97_otu_table_vs.txt --to-tsv --header-key taxonomy

Traceback (most recent call last):
  File "/usr/local/lib/miniconda3/bin/biom", line 11, in <module>
    sys.exit(cli())
  File "/usr/local/lib/miniconda3/lib/python3.5/site-packages/click/core.py", line 716, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/miniconda3/lib/python3.5/site-packages/click/core.py", line 696, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/miniconda3/lib/python3.5/site-packages/click/core.py", line 1060, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/miniconda3/lib/python3.5/site-packages/click/core.py", line 889, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/miniconda3/lib/python3.5/site-packages/click/core.py", line 534, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/miniconda3/lib/python3.5/site-packages/biom/cli/table_converter.py", line 114, in convert
    table = load_table(input_fp)
  File "/usr/local/lib/minicon

## Using FUNGuild

### so far as I understand, funguild was created by some of the big names in fungal community ecology in order to 
### meet the demand for a functional representation of OTUs within a community.  This has been a major hurdle for the field, since it is widley accepted that microfungi will often switch functional groups within a plant dependent upon environmental conditions, host identity, host health, etc (

### It should be noted that the software is designed to create functional assignments at the Genus level.  It seems that this assignment is based upon a citation from the literature.  As such, the assignment may or may not represent the true functional nature of the organism.  However, at this time it may still be worthwile to include such information.  Without the aid of metatranscriptomics, it may not be possible to assign an OTU to a specific functional class.



In [ ]:
## haven't gotten this to work since I created a new biom table... 12/11/16
## update 2/6/17 - looks like the taxonomy header for my otu table is the problem - when the biom table is converted into a text document, 
## the taxonomy appears as distinct headers, whereas the python script requires a header as 'taxonomy'
## to be proporly formatted.  This is the reason why taxonomy was undefined in the error message - we'll try it again!

In [3]:
## I'm trying to figure out whether the script was designed to work with python 2 or 3.   
# python 2 I will need a different shebang line:  #! /usr/local/env python2.7
# python 3:  #! /usr/bin/env python3
# then again, perhaps by adding the prefix 'python' to the command, the shebang is unnessesary.

## command to create Guild classification:
#/usr/local/env python2.7
## to make the script work, I needed to conver the otu_table.txt file into a csv
python2.7 funguild_v1.0.py -otu /Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.csv -db fungi -m -u

## parameters:
##    -otu: required. specify path of otu.txt file 
##    -db: optional.  Choose between fungal (default) and nematode databases
##    -m: optional.  return an additional file containing only otus that have been assigned a function. output sufix _assigned/.txt
##    -u: optional.  Return an additional file containing OTUs that were unassigned.  output sufix _unassigned.txt
## The output file from this command will be labled '97_otu_table.function' 

grahambailes[d93-160:alt_biom]$ python2.7 funguild_v1.0.py -otu /Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.csv -db fungi -m -u
FunGuild v1.0 Beta
Downloading fungi database ...

Reading in the OTU table: '/Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.csv'

Searching the FUNGuild database...
10%
20%
30%
40%
50%
60%
70%
80%
90%
100%

Found 29640 matching taxonomy records in the database.
Dereplicating and sorting the result...
FunGuild tried to assign function to 3093 OTUs in '/Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.csv'.
FUNGuild made assignments on 1868 OTUs.
Result saved to '/Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.guilds.txt'

Additional output:
FUNGuild made assignments on 1868 OTUs, these have been saved to /Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.guilds_matched.txt.
1225 OTUs were unassigned, these are saved to /Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.guilds_unmatched.txt.

Total calculating time: 194.19 seconds.

In [ ]:
## after trimming 
# performed 2019-03-01

grahambailes[Grahams-MacBook-Pro:alt_biom]$ python Guilds_v1.1.py -otu /Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.csv -db fungi -m -u
FunGuild v1.0 Beta
Connecting with FUNGuild database ...

Reading in the OTU table: '/Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.csv'

Searching the FUNGuild database...
10%
20%
30%
40%
50%
60%
70%
80%
90%
100%

Found 4374 matching taxonomy records in the database.
Dereplicating and sorting the result...
FunGuild tried to assign function to 3713 OTUs in '/Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.csv'.
FUNGuild made assignments on 2844 OTUs.
Result saved to '/Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.guilds.txt'

Additional output:
FUNGuild made assignments on 2844 OTUs, these have been saved to /Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.guilds_matched.txt.
869 OTUs were unassigned, these are saved to /Users/grahambailes/grass_endophyte_community/alt_biom/grass_biom_vs_fun.guilds_unmatched.txt.

Total calculating time: 29.7 seconds.

### okay, so The program matched about 61% of my OTUs (1868 of 3093) with some level of functional assignment.  The script returned 1) my original biom.txt file with appended functions, 2) a file containing only those OTUs assigned, and 3) a file containing OTUs that were not assigned.  

### a few notes on 
#### confidence:  The authors note that for the most part, assignment confidence is based upon the primary literature.  these often receive the highest ranking ('highly probably').  Otherwize, the ranking comes from authoritative websites or personal experience from the authors.  There is however one large caviate:  Many fungi can belong to multiple functional guilds, dependent upon life stage, environment, host, etc.  In these cases, the ranking was reduced to possible, to reflect the uncertainty given the taxonomy alone.

## a little frustration continues - The way I set up my taxonomy may have accidentially created a new line break every other line, making the fine unreadable in r.  

I needed to find a way to remove the linebreak, and merge the lines which were now seperated.
After much trial and error, I came upon the sed command:

sed 'N;s/\n/ /'  grass_biom_vs_fun.guilds_matched-2.txt > grass_biom_vs_funguilds_matched.txt

essentially, the command merges line 1&2, 3&4, 5&6, ect, and removes the line break that was accidentially inserted.  As such, I needed to remove the header (line one), and re-insert it later.  I fed sed my misbehaving file, and asked it to write a new one with the correct formating.

remember, sed is a great, versitile tool!

In [ ]:
## at this point, I'm going to import the assigned OTU table into R to have a look - the plan is to exclude those species below the confidence of probable
## to eventually describe community functional differences among sites.